In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
metadata = pd.read_csv("../data/metadata_2021_04_08.tsv", sep="\t", parse_dates = ["Collection date", "Submission date"])

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
metadata.columns

Index(['Virus name', 'Type', 'Accession ID', 'Collection date', 'Location',
       'Additional location information', 'Sequence length', 'Host',
       'Patient age', 'Gender', 'Clade', 'Pango lineage', 'Pangolin version',
       'Variant', 'AA Substitutions', 'Submission date', 'Is reference?',
       'Is complete?', 'Is high coverage?', 'Is low coverage?', 'N-Content',
       'GC-Content'],
      dtype='object')

In [3]:
aadata = metadata[["Accession ID", \
                   "Collection date", \
                   "Submission date", \
                   "Location", \
                   "Additional location information", \
                   "Sequence length", \
                   "Host", \
                   "AA Substitutions", \
                   "Is reference?"]]
def get_nth_slash(row, n):
    try:
        return row.split("/")[n].strip()
    except:
        return np.nan
    
for i in range(4):
    aadata[f"Location_{i}"] = aadata["Location"].apply(lambda row: get_nth_slash(row, i))

/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
metadata[metadata["Pango lineage"] == "B.1.1.7"]["AA Substitutions"].value_counts().reset_index().to_numpy()

array([['(Spike_H69del,NS8_Q27stop,NSP3_T183I,Spike_T716I,NSP6_S106del,N_R203K,Spike_A570D,Spike_N501Y,NSP3_I1412T,NS8_R52I,Spike_P681H,Spike_Y144del,NSP6_G107del,NSP3_A890D,Spike_D1118H,NSP6_F108del,NS8_Y73C,N_G204R,Spike_V70del,NSP12_P323L,Spike_D614G,N_D3L,Spike_S982A,N_S235F)',
        8623],
       ['(Spike_H69del,NS8_Q27stop,NSP3_T183I,Spike_T716I,NSP6_S106del,N_R203K,Spike_A570D,Spike_N501Y,NSP3_I1412T,NS8_R52I,Spike_P681H,Spike_Y144del,NSP2_L550F,NSP6_G107del,NSP3_A890D,Spike_D1118H,NSP6_F108del,NS8_Y73C,N_G204R,Spike_V70del,NSP12_P323L,Spike_D614G,N_D3L,Spike_S982A,N_S235F)',
        4173],
       ['(Spike_H69del,NS8_Q27stop,NSP3_T183I,Spike_T716I,NSP6_S106del,N_R203K,Spike_A570D,NSP13_K460R,Spike_N501Y,NSP3_I1412T,NS8_R52I,Spike_P681H,Spike_Y144del,NSP6_G107del,NSP3_A890D,Spike_D1118H,NSP6_F108del,NS8_Y73C,N_G204R,Spike_V70del,NSP12_P323L,Spike_D614G,N_D3L,Spike_S982A,N_S235F)',
        3690],
       ...,
       ['(Spike_H69del,NS8_Q27stop,NSP3_T183I,Spike_T716I,NSP6_S106del,

In [16]:
i = "NS7a_ins75LHT"
position_finder = re.compile("[0-9]+")
position_finder.search(i)[1]

'75'

In [46]:
aadata[aadata.Location_3 == "Houston"].iloc[0]

Accession ID                                                          EPI_ISL_542487
Collection date                                                  2020-04-10 00:00:00
Submission date                                                  2020-09-23 00:00:00
Location                                       North America / USA / Texas / Houston
Additional location information                                                  NaN
Sequence length                                                                29782
Host                                                                           Human
AA Substitutions                   (NSP14_G6R,NSP14_D496Y,N_R203K,N_G204R,NSP12_P...
Is reference?                                                                    NaN
Location_0                                                             North America
Location_1                                                                       USA
Location_2                                                       

In [ ]:
def get_mutant_info(mutation)


In [39]:
right_weirds = []
left_weirds = []

position_finder = re.compile("[0-9]+")
mutant_list = dict()
# for mutants, count in aadata[aadata.Location_3 == "Houston"]["AA Substitutions"].value_counts().reset_index().to_numpy():
for mutants, count in metadata["AA Substitutions"].value_counts().reset_index().to_numpy():
    mutants = mutants.replace("(", "").replace(")", "").split(",")
    for mutant in mutants:
        if mutant.strip() == "":
            continue
#         if "ins" in mutant:
#             print(mutant)
#             continue
        protein, mutation = mutant.split("_")
        mutation = mutation.replace("stop", "*")
        try:
            position = position_finder.search(mutation)[0]
        except:
            print(mutation)
            continue
        left_mutation, right_mutation = mutation.split(position)
        if len(right_mutation) > 0:
            right_weirds.append(right_mutation)
        if len(left_mutation) > 0:
            left_weirds.append(left_mutation)
        mutant_list.setdefault(protein, []).append((int(position), left_mutation, right_mutation))
#     break

In [40]:
np.unique(left_weirds)

array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'ins'], dtype='<U3')

In [42]:
np.unique(right_weirds)

array(['*', '**', '**E', '**HAL', '**IQ', '**MGSHY**YVR', '**QPS*NTPT',
       '**R*L*', '**RSCTRYSCF', '**S', '**YV', '*A', '*A*G*', '*C*',
       '*CC*T', '*CKQI*CCYY', '*CL', '*CS', '*CS*SIHH', '*CVL*H', '*D',
       '*E', '*EICC*GF*', '*ELYILVLVL', '*ESLD', '*F**',
       '*F*WLFKTY*QCIH*KCRHCGR', '*FFFRL', '*FTVMY*', '*G*Y', '*GF*',
       '*GLE', '*GY*QYHAS*I**LYSY*', '*H*', '*HGLLL*Q', '*HK', '*HLQ',
       '*HLWHCL*KT', '*HQF', '*I', '*I*H*SAI*P', '*I*KI', '*I*V',
       '*IIQIL', '*ILFA', '*ILL', '*IM', '*IP*TPIL', '*IT', '*IWKKL',
       '*K', '*KD', '*KI', '*KNI*VT', '*KY', '*L', '*L*', '*L*T*',
       '*LAY*LLRLLYTLQ*CNFFNCH', '*LDV', '*LH', '*LIL', '*LKVNLQ', '*LL',
       '*LLVFMIT*FLHTM', '*LNFH*', '*LP**', '*LQ', '*LR', '*LRL*SVYD*',
       '*LS', '*LSCRVV', '*LSLIDFYLC', '*LSTSAL', '*LVI', '*LVVMQS*LG',
       '*LY*PS', '*LYV', '*M', '*M**NYTC', '*MEFH', '*ML*I', '*N',
       '*NCRWTN', '*NE', '*NFM', '*NQQWLL', '*NSMS', '*NSMSFVL*', '*NYGL',
       '*PLT', '*Q', '*QAQ

In [38]:
[(key, min([i[0] for i in mutant_list[key]]), max([i[0] for i in mutant_list[key]])) for key in mutant_list.keys()]

[('N', 1, 419),
 ('NSP12', 5, 931),
 ('Spike', 1, 1273),
 ('NS8', 1, 121),
 ('NSP3', 1, 1944),
 ('NSP6', 1, 289),
 ('NS3', 1, 274),
 ('NSP2', 1, 637),
 ('NSP13', 1, 601),
 ('NSP4', 1, 500),
 ('NS7b', 1, 43),
 ('NS7a', 1, 121),
 ('NSP5', 1, 306),
 ('NSP16', 1, 298),
 ('NSP14', 1, 527),
 ('NSP15', 2, 346),
 ('M', 1, 222),
 ('NSP8', 1, 197),
 ('NSP7', 1, 82),
 ('NSP9', 1, 113),
 ('NS6', 1, 61),
 ('NSP1', 1, 179),
 ('NSP10', 1, 139),
 ('E', 1, 75),
 ('NSP11', 1, 13)]